Net stuff

In [117]:
import httpx
import asyncio
from fredapi import Fred
import requests
import numpy as np
import json
import pandas as pd
import pickle
import yahoo_fin.stock_info as si
# We use normal datetime for fred info and pandas datetime for data
from datetime import datetime
from dateutil.relativedelta import relativedelta
from itertools import islice
import math
import os
import matplotlib.pyplot as plt
from functools import lru_cache
from scipy.stats import hmean
from itertools import product
import sys
import cProfile
import yfinance as yf
import warnings
from collections import defaultdict
import traceback

In [2]:
warnings.simplefilter(action='ignore', category=FutureWarning)

In [6]:
modules_to_reload = ["extract_functions", "reshape", "conversions", "net_stuff", "cleanup"]
for module_name in modules_to_reload:
    if module_name in sys.modules:
        del sys.modules[module_name]
    __import__(module_name)
    module = sys.modules[module_name]
    globals().update({name: getattr(module, name) for name in dir(module) if not name.startswith('_')})

In [78]:
#NetCashProvidedByUsedInOperatingActivities NetCashProvidedByUsedInOperatingActivitiesContinuingOperations"WeightedAverageNumberOfSharesOutstandingBasic", "WeightedAverageNumberOfDilutedSharesOutstanding", EntityCommonStockSharesOutstanding
#"static":["Assets", "Liabilities", "AssetsCurrent", "AssetsNoncurrent"],"dynamic":["FreeCashFlow","GrossProfit"]
comp = comp_load("MSFT")
paths = recursive_date_gather(comp,"AccountsReceivableNet", "static")
print(paths)
frame, unit = path_selector(comp,"AccountsReceivableNet",paths["paths"][0], True, dynamic_tolerance=pd.Timedelta(days=1000), static_tolerance=pd.Timedelta(days=1000), reshape_approx = False, forced_dynamic=True)
# frame.to_csv("..\examples/HardearneCOGS9.csv")
print(frame)
# frame.to_csv("..\examples\\realAAPLgross.csv")


{'del': False, 'paths': ({'concat': [({'AccountsReceivableNet': {'add': {'AccountsReceivableNetCurrent': True, 'AccountsReceivableNetNoncurrent': True}}}, [(Timestamp('2017-06-30 00:00:00'), Timestamp('2023-12-31 00:00:00'))])]}, [(Timestamp('2017-06-30 00:00:00'), Timestamp('2023-12-31 00:00:00'))], ['AccountsReceivableNetCurrent', 'AccountsReceivableNetNoncurrent']), 'ignored': []}
            AccountsReceivableNet-0  AccountsReceivableNet-1  \
2017-06-30             2.413100e+10                      NaN   
2017-09-30             1.616100e+10             2.413100e+10   
2017-12-31             2.002800e+10             1.616100e+10   
2018-03-31             1.880800e+10             2.002800e+10   
2018-06-30             2.828100e+10             1.880800e+10   
2018-09-30             1.919000e+10             2.828100e+10   
2018-12-31             2.158000e+10             1.919000e+10   
2019-03-31             2.126900e+10             2.158000e+10   
2019-06-30             3.172400e+10  

In [61]:
comp = comp_load("MSFT")
for i,datapoint in enumerate(list(comp.data['AccountsReceivableNet']["units"].items())[0][1]):
    print(datapoint)

{'end': '2016-07-01', 'val': 22300000000, 'accn': '0001564590-18-019062', 'fy': 2018, 'fp': 'FY', 'form': '10-K', 'filed': '2018-08-03', 'frame': 'CY2016Q2I'}


In [ ]:
company_wordsearch("GOOGL", "short")

Datapoint reshape

In [ ]:
comp = comp_load("GS")
data= comp.converted_data["StockholdersEquity"]
reshaped, intervals, dynamic = reshape('StockholdersEquity', data, comp.ticker, converted = True, use_precompute=False, approx=True, forced_dynamic=True)
print(intervals)
for key,value in list(reshaped.items())[:]:
        print(key,value[0])

Functions

In [200]:
FACTS_PATH = r"C:\Edgar_data"
SUBMISSIONS_PATH = r"C:\Submissions_data"

#Unavailable stuff
with open(r'..\other_pickle\unavailable.json', 'r') as file:
        Unavailable_Measures = json.load(file)

#Lookup table for the undeprecated version of a measure
with open(r"..\other_pickle\deprecated_to_current.json", "r") as file:
    deprecate_conversion = json.load(file)
    file.close()

#Categories
with open(r"..\categories\categories.json", "r") as file:
    categories = json.load(file)

#Irrelevants
with open(r"..\categories\category_measures_irrelevant.json") as file:
    irrelevants = json.load(file)

#Make sure the necessary folders exist
for category, num_range in categories.items():
    os.makedirs(f"..\\companies_data\static\{category}", exist_ok=True)
    os.makedirs(f"..\\companies_data\dynamic\{category}", exist_ok=True)
    os.makedirs(f"..\\clean_data\static\{category}", exist_ok=True)
    os.makedirs(f"..\\clean_data\dynamic\{category}", exist_ok=True)
    os.makedirs(f"..\\companies_data_missing\{category}", exist_ok=True)
    os.makedirs(f"..\clean_data\per_share\static\{category}", exist_ok=True)
    os.makedirs(f"..\clean_data\per_share\dynamic\{category}", exist_ok=True)
    
os.makedirs(f"..\\companies_data\price", exist_ok=True)
os.makedirs(f"..\\clean_data\price", exist_ok=True)
os.makedirs(f"..\\ready_data\static", exist_ok=True)
os.makedirs(f"..\\ready_data\dynamic", exist_ok=True)
os.makedirs(f"..\\ready_data\\indicator", exist_ok=True)
os.makedirs(f"..\\ready_data\\together", exist_ok=True)

with open(r"..\other_pickle\frame_availability.pkl", "rb") as file:
    company_frames_availability = pickle.load(file)

for ticker, available in company_frames_availability.items():
    os.makedirs(f"..\\reshaped\{ticker}", exist_ok=True)

for path in ["checkout", "companies", "companies_data", "companies_data_missing", "units-checkout"]:
    os.makedirs("..\\" + path, exist_ok=True)

with open("..\categories\category_measures.json", "r") as file:
    category_measures = json.load(file)

#Get the measures that you will attempt to get 
with open("..\other_pickle\measures.json", "r") as file:
    Measures = json.load(file)

#The first entry date into the EDGAR database
START = datetime.strptime('1993-01-01', r"%Y-%m-%d")

#Headers for EDGAR call
headers = {
    "User-Agent":"ficakc@seznam.cz",
    "Accept-Encoding":"gzip, deflate",
}

# cik_url =  "https://www.sec.gov/files/company_tickers.json"
# cikdata = requests.get(cik_url, headers=headers).json()

with open(r"..\other_pickle\cik.json","r") as file:
    cikdata = json.load(file)

months_dict = {1: 90, 2: 92, 3: 91, 4: 92, 5: 92, 6: 92, 7: 92, 8: 91, 9: 92, 10: 92, 11: 90, 12: 90}

Indicator frame


In [15]:
indicators = ["TB3MS", "DCOILWTICO","CORESTICKM159SFRBATL"]
indicator_frame = fred_info(indicators, START, datetime.now().date())
indicator_frame.to_csv('..\other_pickle\indicator_frame.csv')

Data grab

In [93]:
#write out measures based on importance in descending order
with open("..\other_pickle\measures.json", "r") as file:
    fund_measures = json.load(file)
#Load out the indicators 
with open("..\other_pickle\indicator_frame.pkl", "rb") as file:
    indicator_frame = pickle.load(file)

#Load the info about the companies that we already have (ticker,edgar,yahoo)
with open(r"..\other_pickle\frame_availability.pkl", "rb") as file:
    company_frames_availability = pickle.load(file)

#Load the unavailable data so far 
with open(r'..\other_pickle\unavailable.json', 'r') as file:
        Unavailable_Measures = json.load(file)

edgar_client =  httpx.AsyncClient()
sem_edgar = asyncio.Semaphore(9)
#Separate sem for yahoo to spread the work and connections
sem_yahoo = asyncio.Semaphore(40)

#Create tasks to ge the first companies_num companies by valuation
# company_frames_availability = {"LLY":((0,0),0)}
companies_num = 800
ticker_dict = dict(islice(company_frames_availability.items(), companies_num))
ticker_list = ticker_fill(ticker_dict)
tasks = []
for ticker in ticker_list:
    tasks.append(async_task(ticker, edgar_client, sem_edgar, sem_yahoo, fund_measures))

availability_list = await asyncio.gather(*tasks)

for ping in availability_list:
    try:
        ticker, avail = ping
        company_frames_availability[ticker] = avail
        if avail == "del":
            company_frames_availability[ticker] = ["del", "del"]
    except Exception as e:
         company_frames_availability[ticker] = ["del", "del"]

success_rate(availability_list)

print(company_frames_availability)
with open(r"..\other_pickle\frame_availability.pkl", "wb") as file:
    pickle.dump(company_frames_availability, file)

#Save the unavailable measures for later use
with open(r'..\other_pickle\unavailable.json', 'w') as file:
        json.dump(Unavailable_Measures,file, indent= 1)

Loading PEG
PEG:45.17122986178361%
Price pinging PEG$
Loading DG
No USD like unit available for DG : dict_keys(['InterestRateSwapAgre', 'swap'])
DG:48.99012567324955%
Price pinging DG$
Loading HPQ
HPQ:44.76260043827611%
Price pinging HPQ$
Loading DVN
No USD like unit available for DVN : dict_keys(['MMBoe'])
No USD like unit available for DVN : dict_keys(['MMBoe'])
No USD like unit available for DVN : dict_keys(['MMBoe'])
No USD like unit available for DVN : dict_keys(['MMBoe'])
No USD like unit available for DVN : dict_keys(['MMBoe'])
No USD like unit available for DVN : dict_keys(['MMBoe'])
No USD like unit available for DVN : dict_keys(['MMBoe'])
No USD like unit available for DVN : dict_keys(['MMBoe'])
No USD like unit available for DVN : dict_keys(['MMBoe'])
No USD like unit available for DVN : dict_keys(['USD/Boe'])
No USD like unit available for DVN : dict_keys(['MMBbls'])
DVN:42.096155293055446%
Price pinging DVN$
Loading WBD
WBD:44.77461791449023%
Price pinging WBD$
Loading TTW

In [151]:
with open(r"..\other_pickle\frame_availability.pkl", "rb") as file:
    company_frames_availability = pickle.load(file)

total = 0
for company, values in list(cikdata.items())[:800]:
    avail = company_frames_availability[values["ticker"]]
    if avail[0][0] and avail[0][1] and avail[1] and avail[1] != "del":
        comp = comp_load(values["ticker"])
        if comp.share_availability[0] or comp.share_availability[1]:
            if comp.missing["dynamic"] == [] and comp.missing["static"] == [] :
                total+= 1
                # print(values["ticker"], comp.share_availability)
    # if avail[0] == "del":
    #     total += 1
print(total)

233


Resets

In [10]:
#BIG RESET!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
#GATHER THE FIRST companies_num companies ciks and pass them to the gather with the tasks
with open(r"..\other_pickle\frame_availability.pkl", "rb") as file:
    company_frames_availability = pickle.load(file)

for company, values in list(cikdata.items())[:]:
    company_frames_availability[values["ticker"]]=[(0,0),0]

print(company_frames_availability)
with open(r"..\other_pickle\frame_availability.pkl", "wb") as file:
    pickle.dump(company_frames_availability, file)


{'AAPL': [(0, 0), 0], 'MSFT': [(0, 0), 0], 'GOOGL': [(0, 0), 0], 'AMZN': [(0, 0), 0], 'NVDA': [(0, 0), 0], 'META': [(0, 0), 0], 'TSLA': [(0, 0), 0], 'BRK-B': [(0, 0), 0], 'LLY': [(0, 0), 0], 'TSM': [(0, 0), 0], 'V': [(0, 0), 0], 'AVGO': [(0, 0), 0], 'JPM': [(0, 0), 0], 'UNH': [(0, 0), 0], 'NVO': [(0, 0), 0], 'SPY': [(0, 0), 0], 'WMT': [(0, 0), 0], 'LVMUY': [(0, 0), 0], 'XOM': [(0, 0), 0], 'MA': [(0, 0), 0], 'JNJ': [(0, 0), 0], 'LTMAY': [(0, 0), 0], 'HD': [(0, 0), 0], 'PG': [(0, 0), 0], 'ASML': [(0, 0), 0], 'COST': [(0, 0), 0], 'ORCL': [(0, 0), 0], 'CVX': [(0, 0), 0], 'MRK': [(0, 0), 0], 'ABBV': [(0, 0), 0], 'ADBE': [(0, 0), 0], 'BAC': [(0, 0), 0], 'CRM': [(0, 0), 0], 'KO': [(0, 0), 0], 'TM': [(0, 0), 0], 'AMD': [(0, 0), 0], 'FMX': [(0, 0), 0], 'PEP': [(0, 0), 0], 'ACN': [(0, 0), 0], 'NFLX': [(0, 0), 0], 'MCD': [(0, 0), 0], 'SHEL': [(0, 0), 0], 'INTC': [(0, 0), 0], 'AZN': [(0, 0), 0], 'TMO': [(0, 0), 0], 'NVS': [(0, 0), 0], 'CSCO': [(0, 0), 0], 'LIN': [(0, 0), 0], 'BABA': [(0, 0), 0], '

In [172]:
with open(r"..\other_pickle\frame_availability.pkl", "rb") as file:
    company_frames_availability = pickle.load(file)
#Frames loaded reset
company_frames_computed = {}
for ticker, availability in company_frames_availability.items():
    if availability[0] == "del":
        company_frames_computed[ticker]= "del"
    else:
        company_frames_computed[ticker]=[0,0,0]
print(company_frames_computed)
with open(r"..\other_pickle\frames_computed_availability.pkl", "wb") as file:
    pickle.dump(company_frames_computed, file)

{'AAPL': [0, 0, 0], 'MSFT': [0, 0, 0], 'GOOGL': [0, 0, 0], 'AMZN': [0, 0, 0], 'NVDA': [0, 0, 0], 'META': 'del', 'TSLA': [0, 0, 0], 'BRK-B': [0, 0, 0], 'LLY': [0, 0, 0], 'TSM': 'del', 'V': [0, 0, 0], 'AVGO': [0, 0, 0], 'JPM': 'del', 'UNH': 'del', 'NVO': 'del', 'SPY': 'del', 'WMT': [0, 0, 0], 'LVMUY': 'del', 'XOM': [0, 0, 0], 'MA': [0, 0, 0], 'JNJ': [0, 0, 0], 'LTMAY': 'del', 'HD': [0, 0, 0], 'PG': [0, 0, 0], 'ASML': 'del', 'COST': 'del', 'ORCL': [0, 0, 0], 'CVX': 'del', 'MRK': 'del', 'ABBV': [0, 0, 0], 'ADBE': [0, 0, 0], 'BAC': [0, 0, 0], 'CRM': [0, 0, 0], 'KO': [0, 0, 0], 'TM': 'del', 'AMD': [0, 0, 0], 'FMX': 'del', 'PEP': 'del', 'ACN': [0, 0, 0], 'NFLX': [0, 0, 0], 'MCD': [0, 0, 0], 'SHEL': [0, 0, 0], 'INTC': [0, 0, 0], 'AZN': 'del', 'TMO': [0, 0, 0], 'NVS': [0, 0, 0], 'CSCO': [0, 0, 0], 'LIN': [0, 0, 0], 'BABA': 'del', 'PDD': [0, 0, 0], 'ABT': [0, 0, 0], 'TMUS': 'del', 'SAP': 'del', 'WFC': [0, 0, 0], 'CMCSA': [0, 0, 0], 'INTU': [0, 0, 0], 'BHP': 'del', 'DHR': [0, 0, 0], 'HDB': 'del',

In [92]:
#UNAVAILABLE RESET
with open(r"..\categories\categories.json", "r") as file:
      categories = json.load(file)
Unavailable_Measures = {}
for category, num_range in categories.items():
    Unavailable_Measures[category] = {}
with open(r'..\other_pickle\unavailable.json', 'w') as file:
        json.dump(Unavailable_Measures,file, indent = 1)

Missing compile

In [93]:
healthy = 0
total = 0
infected = []
for ticker, availability in company_frames_availability.items():
    if availability[0] == "del":
        continue
    (static, dynamic) = availability[0]
    yahoo = availability[1]
    if (static or dynamic) and yahoo == True:
        total += 1
        comp = comp_load(ticker)
        # print(comp.ticker)
        if comp.missing['static'] != [] or comp.missing['dynamic'] != []:
            catandticker =  comp.ticker +" - "+ comp.sic_desc
            cat = get_category(comp.sic)
            Unavailable_Measures[cat][catandticker] = comp.missing['static']
            Unavailable_Measures[cat][catandticker] += comp.missing['dynamic']
            infected.append(comp.ticker)
        else:
            healthy += 1

print(f"{100*healthy/total}%")
    
with open(r'..\other_pickle\unavailable.json', 'w') as file:
    json.dump(Unavailable_Measures,file, indent= 1)



53.28947368421053%


In [154]:
for module in ["extract_functions", "reshape", "conversions", "net_stuff","cleanup"]:
    if module in sys.modules:
        del sys.modules[module]
    import extract_functions
    from extract_functions import *

Get the frames 

In [193]:
with open("..\other_pickle\indicator_frame.pkl", "rb") as file:
    indicator_frame = pickle.load(file)

with open(r"..\other_pickle\frames_computed_availability.pkl", "rb") as file:
    company_frames_computed_availability = pickle.load(file)

#Load the info about the companies that we already have (ticker,edgar,yahoo)
with open(r"..\other_pickle\frame_availability.pkl", "rb") as file:
    company_frames_availability = pickle.load(file)

with open(r'..\other_pickle\unavailable.json', 'r') as file:
        Unavailable_Measures = json.load(file)

with open(r"..\other_pickle\measures.json", "r") as file:
    measures = json.load(file)

# measures = {"static":["Assets", "Liabilities", "AssetsCurrent", "AssetsNoncurrent", "InventoryNet"],"dynamic":["FreeCashFlow","GrossProfit"]}
company_frames_availability = {"ABBV":[(1,1),1]}

computed_iterator = list(company_frames_computed_availability.values())
fuckers = 0
shit_leopards = ["GS"]
stringers = 0
empty_prices = 0
empties = 0
called = 0
tracebackers = 0
missies = 0
i = -1
for ticker, availability in list(company_frames_availability.items())[:800]:
    i += 1
    if availability[0] == "del":
        continue 
    (static, dynamic), yahoo = availability
    if (static or dynamic) and yahoo:
        # if ((not computed_iterator[i][0]) or (not computed_iterator[i][1]) or (not computed_iterator[i][2])) and (computed_iterator[i] != "del"): #Check if we don't have the frames already loaded
            called +=1 
            comp = comp_load(ticker)
            catg = get_category(comp.sic)
            print(f"{i} Getting {comp.ticker} in {catg}")
            try:
                #frames is a dict with static and dynamic frames, if static or dynamic is inelligible, it is not in the dict 
                frames = acquire_frame(comp, measures,(static, dynamic), indicator_frame, reshape_approx=True, lookbehind =5, dynamic_tolerance=pd.Timedelta(days=1000), static_tolerance=pd.Timedelta(days=1000), forced_dynamic=True) #category_measures[catg]
                if not frames["static"].empty:
                    if static and comp.missing["static"] == [] or set(comp.missing["static"]).issubset(set(["PropertyPlantAndEquipmentNet"])):
                        frames["static"].to_csv(f"..\companies_data\static\{catg}\{ticker}.csv")
                        company_frames_computed_availability[ticker][0] = 1
                    else:
                        missies += 1
                else:
                    empties+=1
                if not frames["dynamic"].empty:
                    if dynamic and comp.missing["dynamic"] == []:
                        frames["dynamic"].to_csv(f"..\companies_data\dynamic\{catg}\{ticker}.csv")
                        company_frames_computed_availability[ticker][1] = 1
                    else:
                        missies += 1
                else:
                    empties+=1
                #We also get the aditional columns 
                price_frame = get_label_columns(comp)
                if not price_frame.empty:
                    price_frame.to_csv(f"..\companies_data\price\{ticker}.csv")
                    company_frames_computed_availability[ticker][2] = 1
                else:
                    empty_prices +=1
                # Save it again because of time_init and also save some memory
                with open(f'..\companies\{ticker}.pkl', 'wb') as file:
                    pickle.dump(comp,file)
                    del comp
                print(f"{ticker} saved.")  
            except TypeError as e:
                print(f"{ticker} was supposed to be deleted")
                stringers += 1
            except KeyError as e:
                print(f"Key error {e}")
                exception_traceback = traceback.format_exc()
                tracebackers += 1
            except Exception as e:
                if ticker in shit_leopards:
                    print("Shit leopard")
                else:
                    print(f"{ticker} is fucked in the head: {e}")
                    fuckers += 1    
                company_frames_computed_availability[ticker] = "del"
# with open(r"..\other_pickle\frames_computed_availability.pkl", "wb") as file:
#     pickle.dump(company_frames_computed_availability, file)
print(fuckers, stringers)
total = 0
for ticker, availability in company_frames_computed_availability.items():
    if availability !="del" and availability[0] and availability[1] and availability[2]:
        total+=1
print(total, empty_prices, empties, missies)
print(called, tracebackers)

0 Getting ABBV in Household and Property consumables
ABBV Getting AccountsReceivableNet
{'AccountsReceivableNet': {'add': {'AccountsReceivableNetCurrent': {'concat': [(True, [(Timestamp('2015-12-31 00:00:00'), Timestamp('2023-09-30 00:00:00'))]), ({'AccountsAndOtherReceivablesNetCurrent': True}, [(Timestamp('2012-12-31 00:00:00'), Timestamp('2015-12-31 00:00:00'))])]}}}}
ABBV Getting Assets
{'concat': [({'Assets': True}, [(Timestamp('2012-12-31 00:00:00'), Timestamp('2023-09-30 00:00:00'))])]}
ABBV Getting AssetsCurrent
{'AssetsCurrent': True}
ABBV Getting CashAndCashEquivalentsAtCarryingValue
{'CashAndCashEquivalentsAtCarryingValue': True}
ABBV Getting InventoryNet
{'InventoryNet': True}
ABBV Getting Liabilities
{'concat': [({'Liabilities': {'sub': {'Assets': {'concat': [(True, [(Timestamp('2012-12-31 00:00:00'), Timestamp('2023-09-30 00:00:00'))])]}, 'StockholdersEquity': {'concat': [(True, [(Timestamp('2012-12-31 00:00:00'), Timestamp('2023-09-30 00:00:00'))])]}}}}, [(Timestamp('201

In [176]:
availabels = 0
for ticker, availability in list(company_frames_availability.items())[:50]:
    if availability[1] == "del":
        continue 
    comp = comp_load(ticker)
    print(comp.missing)
    # (static, dynamic), yahoo = availability
    # if (static or dynamic) and yahoo:
    #     availabels+=1
# print(availabels)

{'static': [], 'dynamic': []}
{'static': [], 'dynamic': []}
{'static': [], 'dynamic': []}
{'static': [], 'dynamic': []}
{'static': [], 'dynamic': []}
{'static': [], 'dynamic': []}
{'static': ['AccountsPayableCurrent', 'AccountsReceivableNet', 'AssetsCurrent', 'LiabilitiesCurrent', 'LongTermDebt', 'AssetsNoncurrent', 'PropertyPlantAndEquipmentNet', 'ShortTermBorrowings'], 'dynamic': ['GrossProfit']}
{'static': [], 'dynamic': []}
{'static': [], 'dynamic': ['GrossProfit']}
{'static': [], 'dynamic': []}
{'static': [], 'dynamic': []}
{'static': [], 'dynamic': ['GrossProfit']}
{'static': [], 'dynamic': []}
{'static': ['LongTermDebt'], 'dynamic': []}
{'static': [], 'dynamic': []}
{'static': [], 'dynamic': []}
{'static': ['LongTermDebt'], 'dynamic': []}
{'static': ['AccountsPayableCurrent'], 'dynamic': []}
{'static': [], 'dynamic': []}
{'static': ['AccountsPayableCurrent', 'AccountsReceivableNet', 'AssetsCurrent', 'InventoryNet', 'LiabilitiesCurrent', 'AssetsNoncurrent'], 'dynamic': ['GrossPro

In [178]:
#If you want to save progress after interrupt
with open(r"..\other_pickle\frames_computed_availability.pkl", "wb") as file:
    pickle.dump(company_frames_computed_availability, file)

In [109]:
company_frames_computed_availability["NET"]

'del'

XBRL 

In [ ]:
Elementframe = pd.read_csv("ListOfElements.csv")
# Elementframe = Elementframe[Elementframe["approvalStatus"].apply(lambda rem: rem=="Final")]
# Elementframe.to_csv("ListOfElements.csv")
def xbrl_wordsearch(word, length):
    matches = Elementframe[Elementframe["elementName"].apply(lambda x: x if (len(x)<length) else "").str.contains(word, na=False)]
    if matches.empty:
        print("No match")
        return
    print(json.dumps(matches[["elementName","definition","Deprecated"]].to_dict(orient="records"), indent = 2))

with open(r"companies\MSFT.pkl", "rb") as file:
    company = pickle.load(file)

with open(r"units.json", "w") as file:
    json.dump(compdict,file, indent= 1)

Splits 

In [186]:
tickerSymbol = 'MSFT'
tickerData = yf.Ticker(tickerSymbol)
tickerDf = tickerData.history(period='1d', start='2010-1-1', end='2020-1-1')
tickerDf.to_csv("..\examples\Otherpricemethod")

In [14]:
comp = comp_load("NVDA")
price_frame = get_label_columns(comp)
price_frame.to_csv("..\examples\\new_price.csv")

In [186]:
modules_to_reload = ["extract_functions", "reshape", "conversions", "net_stuff","cleanup"]
for module_name in modules_to_reload:
    if module_name in sys.modules:
        del sys.modules[module_name]
    __import__(module_name)
    module = sys.modules[module_name]
    globals().update({name: getattr(module, name) for name in dir(module) if not name.startswith('_')})

Cleanup

In [18]:
with open(r"..\other_pickle\frames_computed_availability.pkl", "rb") as file:
    company_frames_computed_availability = pickle.load(file)

In [196]:
ignore_columns = ["PropertyPlantAndEquipmentNet"]
with open(r"..\other_pickle\ignored_columns.pkl", "wb") as file:
    pickle.dump(ignore_columns, file)
cleanup(company_frames_computed_availability, limit= 500, tether_index=True, ignore_columns=["PropertyPlantAndEquipmentNet"])

Cleaning AAPL
Cleaning MSFT
Cleaning AMZN
Cleaning NVDA
Cleaning TSLA
Cleaning BRK-B
Cleaning LLY
Cleaning V
Cleaning AVGO
Cleaning WMT
Cleaning XOM
Cleaning MA
Cleaning JNJ
Cleaning HD
Cleaning PG
Cleaning ORCL
Cleaning ABBV
Cleaning ADBE
Cleaning BAC
Cleaning CRM
Cleaning KO
Cleaning AMD
Cleaning ACN
Cleaning NFLX
Cleaning MCD
Cleaning INTC
Cleaning TMO
Cleaning CSCO
Cleaning LIN
Cleaning ABT
Cleaning WFC
Cleaning CMCSA
Cleaning INTU
Cleaning DHR
Cleaning NKE
Cleaning DIS
Cleaning QCOM
Cleaning PFE
Cleaning VZ
Cleaning BA
Cleaning TXN
Cleaning AMGN
Cleaning MS
Cleaning CAT
Cleaning UNP
Cleaning IBM
Cleaning PM
Cleaning NOW
Cleaning SPGI
Cleaning COP
Cleaning HON
Cleaning AMAT
Cleaning UPS
Cleaning UBER
Cleaning LOW
Cleaning SCHW
Cleaning NEE
Cleaning BKNG
Cleaning PLD
Cleaning RTX
Cleaning BLK
Cleaning ISRG
Cleaning SYK
Cleaning LMT
Cleaning ELV
Cleaning MDT
Cleaning SBUX
Cleaning TJX
Cleaning VRTX
Cleaning BMY
Cleaning LRCX
Cleaning CVS
Cleaning GILD
Cleaning AMT
Cleaning C
Cleaning

In [197]:
ghost(company_frames_computed_availability, limit=500, size = 50, adjustment=0.5, iterations = 4, verbose=True)

Ghost for AAPL
Ghost for MSFT
Ghost for AMZN
Ghost for NVDA
Ghost for TSLA
Ghost for BRK-B
Ghost for LLY
Ghost for V
Ghost for AVGO
Ghost for WMT
Ghost for XOM
Ghost for MA
Ghost for JNJ
Ghost for HD
Ghost for PG
Ghost for ORCL
Ghost for ABBV
Ghost for ADBE
Ghost for BAC
Ghost for CRM
Ghost for KO
Ghost for AMD
Ghost for ACN
Ghost for NFLX
Ghost for MCD
Ghost for INTC
Ghost for TMO
Ghost for CSCO
Ghost for LIN
Ghost for ABT
Ghost for WFC
Ghost for CMCSA
Ghost for INTU
Ghost for DHR
Ghost for NKE
Ghost for DIS
Ghost for QCOM
Ghost for PFE
Ghost for VZ
Ghost for BA
Ghost for TXN
Ghost for AMGN
Ghost for MS
Ghost for CAT
Ghost for UNP
Ghost for IBM
Ghost for PM
Ghost for NOW
Ghost for SPGI
Ghost for COP
Ghost for HON
Ghost for AMAT
Ghost for UPS
Ghost for UBER
Ghost for LOW
Ghost for SCHW
Ghost for NEE
Ghost for BKNG
Ghost for PLD
Ghost for RTX
Ghost for BLK
Ghost for ISRG
Ghost for SYK
Ghost for LMT
Ghost for ELV
Ghost for MDT
Ghost for SBUX
Ghost for TJX
Ghost for VRTX
Ghost for BMY
Gho

In [ ]:
quantiles(company_frames_computed_availability, limit=100, quantile = 0.8, period_dict={"static": 30, "dynamic": 90})

In [ ]:
upper_averages(company_frames_computed_availability, limit=100, quantile = 0.8, period_dict={"static": 30, "dynamic": 90})

In [198]:
per_share_divide(company_frames_computed_availability, limit=500)

Dividing AAPL
Dividing MSFT
Dividing AMZN
Dividing NVDA
Dividing TSLA
Dividing BRK-B
Dividing LLY
Dividing V
Dividing AVGO
Dividing WMT
Dividing XOM
Dividing MA
Dividing JNJ
Dividing HD
Dividing PG
Dividing ORCL
Dividing ABBV
Dividing ADBE
Dividing BAC
Dividing CRM
Dividing KO
Dividing AMD
Dividing ACN
Dividing NFLX
Dividing MCD
Dividing INTC
Dividing TMO
Dividing CSCO
Dividing LIN
Dividing ABT
Dividing WFC
Dividing CMCSA
Dividing INTU
Dividing DHR
Dividing NKE
Dividing DIS
Dividing QCOM
Dividing PFE
Dividing VZ
Dividing BA
Dividing TXN
Dividing AMGN
Dividing MS
Dividing CAT
Dividing UNP
Dividing IBM
Dividing PM
Dividing NOW
Dividing SPGI
Dividing COP
Dividing HON
Dividing AMAT
Dividing UPS
Dividing UBER
Dividing LOW
Dividing SCHW
Dividing NEE
Dividing BKNG
Dividing PLD
Dividing RTX
Dividing BLK
Dividing ISRG
Dividing SYK
Dividing LMT
Dividing ELV
Dividing MDT
Dividing SBUX
Dividing TJX
Dividing VRTX
Dividing BMY
Dividing LRCX
Dividing CVS
Dividing GILD
Dividing AMT
Dividing C
Dividing

In [201]:
ready_data(company_frames_computed_availability, limit=500, lookbehind=5, dynamic_shift= pd.Timedelta(days=120), static_shift =pd.Timedelta(days=90), multiples=False, categories=False, averages = False, ghost = True)

Readying AAPL
Readying MSFT
Readying GOOGL
Readying AMZN
Readying NVDA
Readying META
Readying TSLA
Readying BRK-B
Readying LLY
Readying TSM
Readying V
Readying AVGO
Readying JPM
Readying UNH
Readying NVO
Readying SPY
Readying WMT
Readying LVMUY
Readying XOM
Readying MA
Readying JNJ
Readying LTMAY
Readying HD
Readying PG
Readying ASML
Readying COST
Readying ORCL
Readying CVX
Readying MRK
Readying ABBV
Readying ADBE
Readying BAC
Readying CRM
Readying KO
Readying TM
Readying AMD
Readying FMX
Readying PEP
Readying ACN
Readying NFLX
Readying MCD
Readying SHEL
Readying INTC
Readying AZN
Readying TMO
Readying NVS
Readying CSCO
Readying LIN
Readying BABA
Readying PDD
Readying ABT
Readying TMUS
Readying SAP
Readying WFC
Readying CMCSA
Readying INTU
Readying BHP
Readying DHR
Readying HDB
Readying NKE
Readying DIS
Readying QCOM
Readying PFE
Readying QQQ
Readying BX
Readying TTE
Readying VZ
Readying BA
Readying TXN
Readying HSBC
Readying AMGN
Readying MS
Readying CAT
Readying RTNTF
Readying UNP
Re